In [2]:
from google.cloud import bigquery
# Construct a BigQuery client object.
client = bigquery.Client()

/Users/robords/.venv/cloud_weather_predictions/lib/python3.7/site-packages/google/auth/_default.py:81: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
/Users/robords/.venv/cloud_weather_predictions/lib/python3.7/site-packages/google/auth/_default.py:81: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service account

### Get the list of tables in the dataset

In [12]:
dataset_id = 'bigquery-public-data.ghcn_d'
tables = client.list_tables(dataset_id)  # Make an API request.

print("Tables contained in '{}':".format(dataset_id))
tables_in_noaa_dataset = []
for table in tables:
    tables_in_noaa_dataset.append("{}.{}.{}".format(table.project, table.dataset_id, table.table_id))

Tables contained in 'bigquery-public-data.ghcn_d':


### Get the schemas of each table in the dataset

In [13]:
tables_in_noaa_dataset[0]

'bigquery-public-data.ghcn_d.ghcnd_1763'

In [15]:
table_id = 'bigquery-public-data.ghcn_d.ghcnd_2021'
table = client.get_table(table_id)  # Make an API request.

# View table properties
print(
    "Got table '{}.{}.{}'.".format(table.project, table.dataset_id, table.table_id)
)
print("Table schema: {}".format(table.schema))
print("Table description: {}".format(table.description))
print("Table has {} rows".format(table.num_rows))

Got table 'bigquery-public-data.ghcn_d.ghcnd_2021'.
Table schema: [SchemaField('id', 'STRING', 'REQUIRED', '', (), None), SchemaField('date', 'DATE', 'NULLABLE', '', (), None), SchemaField('element', 'STRING', 'NULLABLE', '', (), None), SchemaField('value', 'FLOAT', 'NULLABLE', '', (), None), SchemaField('mflag', 'STRING', 'NULLABLE', '', (), None), SchemaField('qflag', 'STRING', 'NULLABLE', '', (), None), SchemaField('sflag', 'STRING', 'NULLABLE', '', (), None), SchemaField('time', 'STRING', 'NULLABLE', '', (), None), SchemaField('source_url', 'STRING', 'NULLABLE', 'Source ', (), None), SchemaField('etl_timestamp', 'TIMESTAMP', 'NULLABLE', 'Load time for this data row', (), None)]
Table description: None
Table has 37098011 rows


In [ ]:
for table in tables_in_noaa_dataset:
    schema = []
    query = f"""
    SELECT
        * EXCEPT(is_generated, generation_expression, is_stored, is_updatable)
      FROM
        `bigquery-public-data`.census_bureau_usa.INFORMATION_SCHEMA.COLUMNS
      WHERE
        table_name = table;
    """
    query_job = client.query(query)  # Make an API request.

    query_job.to_dataframe()

In [17]:
_2021_data = """
SELECT
  *
FROM
  `bigquery-public-data.ghcn_d.ghcnd_2021`
LIMIT 10
"""
_2021_data_query_job = client.query(_2021_data)  # Make an API request.

_2021_data_query_job.to_dataframe()

,id,date,element,value,mflag,qflag,sflag,time,source_url,etl_timestamp
0,USC00084394,2021-10-23,TMAX,294.0,None,None,H,0800,ftp://ftp.ncdc.noaa.gov/pub/data/ghcn/daily/by...,2022-10-09 03:24:20.113166+00:00
1,USW00054795,2021-10-24,PRCP,0.0,None,None,R,None,ftp://ftp.ncdc.noaa.gov/pub/data/ghcn/daily/by...,2022-10-09 03:24:20.113166+00:00
2,USW00063869,2021-10-24,TMAX,238.0,None,None,R,None,ftp://ftp.ncdc.noaa.gov/pub/data/ghcn/daily/by...,2022-10-09 03:24:20.113166+00:00
3,USC00130046,2021-10-25,PRCP,406.0,None,None,H,None,ftp://ftp.ncdc.noaa.gov/pub/data/ghcn/daily/by...,2022-10-09 03:24:20.113166+00:00
4,USW00054903,2021-10-25,PRCP,0.0,None,None,R,None,ftp://ftp.ncdc.noaa.gov/pub/data/ghcn/daily/by...,2022-10-09 03:24:20.113166+00:00
5,USW00023803,2021-10-23,TMAX,247.0,None,None,R,None,ftp://ftp.ncdc.noaa.gov/pub/data/ghcn/daily/by...,2022-10-09 03:24:20.113166+00:00
6,CAW00064757,2021-10-24,PRCP,0.0,None,None,R,None,ftp://ftp.ncdc.noaa.gov/pub/data/ghcn/daily/by...,2022-10-09 03:24:20.113166+00:00
7,USC00227276,2021-10-24,TMIN,111.0,None,None,H,0730,ftp://ftp.ncdc.noaa.gov/pub/data/ghcn/daily/by...,2022-10-09 03:24:20.113166+00:00
8,USC00329233,2021-10-24,TMIN,17.0,None,None,H,0800,ftp://ftp.ncdc.noaa.gov/pub/data/ghcn/daily/by...,2022-10-09 03:24:20.113166+00:00
9,USW00094078,2021-10-24,PRCP,0.0,None,None,R,None,ftp://ftp.ncdc.noaa.gov/pub/data/ghcn/daily/by...,2022-10-09 03:24:20.113166+00:00


In [1]:
query = """
SELECT
  id,
  name,
  state,
  latitude,
  longitude
FROM
  `bigquery-public-data.ghcn_d.ghcnd_stations`
WHERE
  latitude > 41.7
  AND latitude < 42
  AND longitude > -87.7
  AND longitude < -87.5
"""
query_job = client.query(query)  # Make an API request.

query_job.to_dataframe()

/Users/robords/.venv/cloud_weather_predictions/lib/python3.7/site-packages/google/auth/_default.py:81: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
/Users/robords/.venv/cloud_weather_predictions/lib/python3.7/site-packages/google/auth/_default.py:81: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service account

,id,name,state,latitude,longitude
0,USC00111562,CHICAGO SAN DIST OFC,IL,41.9000,-87.6333
1,US1ILCK0117,CHICAGO 3.0 NW,IL,41.9125,-87.6695
2,US1ILCK0108,BEVERLY 0.3 N,IL,41.7165,-87.6710
3,US1ILCK0104,LINCOLNWOOD 5.1 SE,IL,41.9494,-87.6703
4,US1ILCK0151,EVERGREEN PARK 0.5 NE,IL,41.7274,-87.6949
5,US1ILCK0122,CHICAGO 6.5 NNE,IL,41.9272,-87.6517
6,USC00111557,CHICAGO RACINE PUMP,IL,41.8167,-87.6500
7,US1ILCK0097,CHICAGO 6.8 NNE,IL,41.9301,-87.6393
8,USC00111584,CHICAGO WB CITY 2,IL,41.8833,-87.6333
9,US1ILCK0010,CHICAGO 3.0 N,IL,41.8798,-87.6823
